In [ ]:
from copy import copy 
from matplotlib import pyplot as plt
import datetime

import numpy as np
import pandas as pd

import summer2
from summer.utils import ref_times_to_dti

from autumn.core.project import get_project, load_timeseries
from autumn.core.project.params import get_with_nested_key

# Targets represent data we are trying to fit to
from estival import targets as est
# We specify parameters using (Bayesian) priors
from estival import priors as esp
# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

from autumn.projects.sm_covid2.common_school.calibration import get_bcm_object

In [ ]:
bcm = get_bcm_object("france")

### Helper function to plot model fit to data

In [ ]:
def plot_fit(bcm, params):

    REF_DATE = datetime.date(2019,12,31)
    datetime_target = copy(bcm.targets["infection_deaths"].data)
    datetime_target.index = ref_times_to_dti(REF_DATE, datetime_target.index)

    ax = bcm.run(params).derived_outputs["infection_deaths"].plot()
    datetime_target.plot(style='.')
    ll = bcm.loglikelihood(**params)

    text = f"ll={ll}"
    plt.text(0.8, 0.9, text, transform=ax.transAxes)

# Compare model fits

In [ ]:
import numpy as np

In [ ]:
example_params = {
    'contact_rate': 0.04104869232093793,
    'infectious_seed_time': 30.447925441292778,
    'age_stratification.ifr.multiplier': 0.8145117432172231,
    'voc_emergence.delta.new_voc_seed.time_from_gisaid_report': 18.52679245171894,
    'voc_emergence.omicron.new_voc_seed.time_from_gisaid_report': 0.0,
    'random_process.noise_sd': 0.5835215881701278,

    'mobility.unesco_partial_opening_value': .2,
    'school_multiplier': 1.,

    "random_process.delta_values": np.array([
-1.306496185258189,
 0.11250765346332026,
0.5009919549420943,
 -0.06375110616249913,
 -0.24028730546991617,
0.2606697551298094,
-0.13194034829544132,
-0.3559750720412327,
0.04754597441674813,
 0.12138039221640762,
0.6411075116621299,  
        0.0,0.0,0.0,0.0,0.0,0.0,
    ]),

    'infection_deaths_dispersion_param': 100.
}

In [ ]:
plot_fit(bcm, example_params)

In [ ]:
from estival.optimization import nevergrad as eng
import nevergrad as ng

In [ ]:
opt = eng.optimize_model(bcm, obj_function=bcm.loglikelihood)

In [ ]:
res = opt.minimize(2000)
print(res.value[1])
plot_fit(bcm, res.value[1])

In [ ]:
plot_fit(bcm, res.value[1])

In [ ]:
res = opt.minimize(6000)
print(res.value[1])
plot_fit(bcm, res.value[1])

### Print params formated for yml file

In [ ]:
for p, val in res.value[1].items():
    if p == "random_process.delta_values":
        for i, deltaval in enumerate(val):
            print(f"random_process.delta_values({i}): {deltaval}")
    else:
        
        print(f"{p}: {val}")

In [ ]:
from estival.calibration import pymc as epm
import pymc as pm

In [ ]:
initvals = res.value[1]

with pm.Model() as model:    
    variables = epm.use_model(bcm)
    idata = pm.sample(step=[pm.DEMetropolis(variables)], draws=100, tune=0,cores=4,chains=4, initvals=initvals)

In [ ]:
# Save pymc outputs to disk
idata.to_netcdf("calib_DATE.nc")